# DEMO: Data Exploration and Feature Engineering

## Imports

In [7]:
from snowflake.snowpark.session import Session
import snowflake.snowpark.functions as F
import snowflake.snowpark.types as T
from snowflake.snowpark.window import Window
from snowflake.ml.modeling.preprocessing import *
from snowflake.ml.modeling.impute import *

import sys
import json
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os

## Create Snowpark Session

In [8]:
os.chdir('/Users/nicholasaverillo/Documents/MachineLearning/sfguide-getting-started-machine-learning/hol/')
print(os.getcwd())
with open('creds.json') as f:
    connection_parameters = json.load(f)

/Users/nicholasaverillo/Documents/MachineLearning/sfguide-getting-started-machine-learning/hol


In [9]:
session = Session.builder.configs(connection_parameters).create()
print(f"Current Database and schema: {session.get_fully_qualified_current_schema()}")
print(f"Current Warehouse: {session.get_current_warehouse()}")

Current Database and schema: "HOL_DB"."PUBLIC"
Current Warehouse: "HOL_WH"


## Snowpark DataFrames vs. Pandas DataFrames

In [10]:
# Creating a Pandas DataFrame
pandas_df = pd.read_csv('data/application_record.csv.zip')
print(type(pandas_df))

<class 'pandas.core.frame.DataFrame'>


In [11]:
# Creating a Snowpark DataFrame
snowpark_df = session.table('APPLICATION_RECORD')
print(type(snowpark_df))

<class 'snowflake.snowpark.table.Table'>


In [12]:
# Compare size
print('Size in MB of Pandas DataFrame in Memory:\n', np.round(sys.getsizeof(pandas_df) / (1024.0**2), 2))
print('Size in MB of Snowpark DataFrame in Memory:\n', np.round(sys.getsizeof(snowpark_df) / (1024.0**2), 2))

Size in MB of Pandas DataFrame in Memory:
 251.15
Size in MB of Snowpark DataFrame in Memory:
 0.0


The only thing stored in a Snowpark DataFrame is the SQL needed to return data

In [13]:
snowpark_df.queries

{'queries': ['SELECT  *  FROM (APPLICATION_RECORD)'], 'post_actions': []}

A Snowpark DataFrame can be converted to a Pandas DataFrame. This will pull the data from Snowflake into the Python enviroment memory.

In [14]:
pandas_df2 = snowpark_df.to_pandas()

Both our Pandas DataFrames has the same shape

In [15]:
pandas_df.shape, pandas_df2.shape

((438557, 18), (438557, 18))

Showing a Snowpark DataFrame

In [16]:
# snowpark_df.show() <- also possible
snowpark_df.limit(5).to_pandas() # <- collects first 5 rows and displays as pandas-dataframe

,ID,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,DAYS_BIRTH,DAYS_EMPLOYED,FLAG_MOBIL,FLAG_WORK_PHONE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS
0,5008804,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,-12005,-4542,1,1,0,0,None,2.0
1,5008805,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,-12005,-4542,1,1,0,0,None,2.0
2,5008806,M,Y,Y,0,112500.0,Working,Secondary / secondary special,Married,House / apartment,-21474,-1134,1,0,0,0,Security staff,2.0
3,5008808,F,N,Y,0,270000.0,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,-19110,-3051,1,0,1,1,Sales staff,1.0
4,5008809,F,N,Y,0,270000.0,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,-19110,-3051,1,0,1,1,Sales staff,1.0


## Simple Transformations
Select specific columns

In [17]:
# snowpark_df = snowpark_df.select('CODE_GENDER','NAME_INCOME_TYPE','DAYS_BIRTH',)
snowpark_df = snowpark_df[['CODE_GENDER','NAME_INCOME_TYPE','DAYS_BIRTH']] # -> pandas-like selection
snowpark_df.show()

-------------------------------------------------------
|"CODE_GENDER"  |"NAME_INCOME_TYPE"    |"DAYS_BIRTH"  |
-------------------------------------------------------
|M              |Working               |-12005        |
|M              |Working               |-12005        |
|M              |Working               |-21474        |
|F              |Commercial associate  |-19110        |
|F              |Commercial associate  |-19110        |
|F              |Commercial associate  |-19110        |
|F              |Commercial associate  |-19110        |
|F              |Pensioner             |-22464        |
|F              |Pensioner             |-22464        |
|F              |Pensioner             |-22464        |
-------------------------------------------------------



To add a new column to a Snowpark DataFrame the **with_column** method can be used.  
In below example we are adding a neew column, AGE, that calculates the number of years that DAYS_BIRTH is.

In [18]:
# Create a new column
# Formula: Absolute Value of DAYS_BIRTH divided by 365 days rounded down
snowpark_df = snowpark_df.with_column('AGE', F.floor(F.abs(F.col('DAYS_BIRTH')) / 365))
snowpark_df.show()

---------------------------------------------------------------
|"CODE_GENDER"  |"NAME_INCOME_TYPE"    |"DAYS_BIRTH"  |"AGE"  |
---------------------------------------------------------------
|M              |Working               |-12005        |32     |
|M              |Working               |-12005        |32     |
|M              |Working               |-21474        |58     |
|F              |Commercial associate  |-19110        |52     |
|F              |Commercial associate  |-19110        |52     |
|F              |Commercial associate  |-19110        |52     |
|F              |Commercial associate  |-19110        |52     |
|F              |Pensioner             |-22464        |61     |
|F              |Pensioner             |-22464        |61     |
|F              |Pensioner             |-22464        |61     |
---------------------------------------------------------------



If we do not want to use specific columns we can use **drop** to remove those from a Snowpark DataFrame.  
**Note:** This is not removing them from the underlying table.

In [19]:
# Drop a column
snowpark_df = snowpark_df.drop('DAYS_BIRTH')
snowpark_df.show()

------------------------------------------------
|"CODE_GENDER"  |"NAME_INCOME_TYPE"    |"AGE"  |
------------------------------------------------
|M              |Working               |32     |
|M              |Working               |32     |
|M              |Working               |58     |
|F              |Commercial associate  |52     |
|F              |Commercial associate  |52     |
|F              |Commercial associate  |52     |
|F              |Commercial associate  |52     |
|F              |Pensioner             |61     |
|F              |Pensioner             |61     |
|F              |Pensioner             |61     |
------------------------------------------------



To filter/select specific rows we use **filter**

In [20]:
# Filter data
snowpark_df = snowpark_df.filter(F.col('NAME_INCOME_TYPE').in_(['Pensioner','Student']))
snowpark_df.show()

----------------------------------------------
|"CODE_GENDER"  |"NAME_INCOME_TYPE"  |"AGE"  |
----------------------------------------------
|F              |Pensioner           |61     |
|F              |Pensioner           |61     |
|F              |Pensioner           |61     |
|F              |Pensioner           |55     |
|F              |Pensioner           |61     |
|F              |Pensioner           |61     |
|F              |Pensioner           |61     |
|F              |Pensioner           |61     |
|F              |Pensioner           |61     |
|F              |Pensioner           |61     |
----------------------------------------------



To aggregate data the **group_by** method are used in combination with the **agg** method.

In [21]:
snowpark_df = snowpark_df.group_by(['CODE_GENDER','NAME_INCOME_TYPE']).agg([F.avg('AGE').as_('AVG_AGE')])
snowpark_df.show()

--------------------------------------------------
|"CODE_GENDER"  |"NAME_INCOME_TYPE"  |"AVG_AGE"  |
--------------------------------------------------
|F              |Pensioner           |59.188624  |
|M              |Pensioner           |57.685482  |
|F              |Student             |46.090909  |
|M              |Student             |27.166667  |
--------------------------------------------------



To sort the data in the dataframe **sort** is used.

In [22]:
# Sort data
snowpark_df = snowpark_df.sort(F.col('AVG_AGE').desc())
snowpark_df.show()

--------------------------------------------------
|"CODE_GENDER"  |"NAME_INCOME_TYPE"  |"AVG_AGE"  |
--------------------------------------------------
|F              |Pensioner           |59.188624  |
|M              |Pensioner           |57.685482  |
|F              |Student             |46.090909  |
|M              |Student             |27.166667  |
--------------------------------------------------



## Simple Data Analysis
In this section we will use API Snowpark to do some basic analysis of our data.  
Start by creating a new Snowpark DataFrame

In [23]:
# Creating a Snowpark DataFrame
snowpark_df = session.table('APPLICATION_RECORD')

The **count** method on a DataFrame will return the number of rows

In [24]:
# Number of rows in dataset
snowpark_df.count()

438557

If we want to filter out duplicated rows, keeping only one, we can use the **drop_duplicates** method.

In [25]:
# Lets drop duplicates based on ID
snowpark_df = snowpark_df.drop_duplicates('ID')
snowpark_df.count()

438510

Duplicated rows are only filtered and we can see the logic for it by examining the SQL for the DataFrame, using ['queries'][0] will return the first SQL statement for the DataFrame

In [26]:
print(snowpark_df.queries['queries'][0])

SELECT "ID", "CODE_GENDER", "FLAG_OWN_CAR", "FLAG_OWN_REALTY", "CNT_CHILDREN", "AMT_INCOME_TOTAL", "NAME_INCOME_TYPE", "NAME_EDUCATION_TYPE", "NAME_FAMILY_STATUS", "NAME_HOUSING_TYPE", "DAYS_BIRTH", "DAYS_EMPLOYED", "FLAG_MOBIL", "FLAG_WORK_PHONE", "FLAG_PHONE", "FLAG_EMAIL", "OCCUPATION_TYPE", "CNT_FAM_MEMBERS" FROM ( SELECT "ID", "CODE_GENDER", "FLAG_OWN_CAR", "FLAG_OWN_REALTY", "CNT_CHILDREN", "AMT_INCOME_TOTAL", "NAME_INCOME_TYPE", "NAME_EDUCATION_TYPE", "NAME_FAMILY_STATUS", "NAME_HOUSING_TYPE", "DAYS_BIRTH", "DAYS_EMPLOYED", "FLAG_MOBIL", "FLAG_WORK_PHONE", "FLAG_PHONE", "FLAG_EMAIL", "OCCUPATION_TYPE", "CNT_FAM_MEMBERS", row_number() OVER (PARTITION BY "ID"  ORDER BY "ID" ASC NULLS FIRST ) AS "HKRGQD0JXU" FROM APPLICATION_RECORD) WHERE ("HKRGQD0JXU" = 1 :: INT)


Using the **describe** method will return some basic statistics for all numeric and string columns.

In [27]:
# Calculating various statistics per column
snowpark_df.describe().show()

--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|"SUMMARY"  |"ID"               |"CODE_GENDER"  |"FLAG_OWN_CAR"  |"FLAG_OWN_REALTY"  |"CNT_CHILDREN"      |"AMT_INCOME_TOTAL"  |"NAME_INCOME_TYPE"    |"NAME_EDUCATION_TYPE"          |"NAME_FAMILY_STATUS"  |"NAME_HOUSING_TYPE"  |"DAYS_BIRTH"       |"DAYS_EMPLOYED"     |"FLAG_MOBIL"  |"FLAG_WORK_PHONE"   |"FLAG_PHONE"         |"FLAG_EMAIL"        |"OCCUPATION_TYPE"     |"CNT_FAM_MEMBERS"   |
----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

Using **group_by** and **agg** alows us to calculate the mean value of AMT_INCOME_TOTAL by NAME_INCOME_TYPE and CODE_GENDER. Using **sort** to return the data ordered by NAME_INCOME_TYPE in ascending order and AVG_INCOME by descending order.

In [28]:
# Average Income per Income Type and Gender
analysis_df = snowpark_df.group_by(['NAME_INCOME_TYPE','CODE_GENDER']).agg([F.mean('AMT_INCOME_TOTAL').as_('AVG_INCOME')])
analysis_df = analysis_df.sort('NAME_INCOME_TYPE', F.col('AVG_INCOME').desc())
analysis_df.show()

-------------------------------------------------------------
|"NAME_INCOME_TYPE"    |"CODE_GENDER"  |"AVG_INCOME"        |
-------------------------------------------------------------
|Commercial associate  |M              |249208.08642289176  |
|Commercial associate  |F              |206579.17463258584  |
|Pensioner             |M              |169049.77416737832  |
|Pensioner             |F              |150729.61255448588  |
|State servant         |M              |237034.15414285715  |
|State servant         |F              |186152.9842904419   |
|Student               |F              |165272.72727272726  |
|Student               |M              |149250.0            |
|Working               |M              |202170.82427397132  |
|Working               |F              |168679.56899413437  |
-------------------------------------------------------------



We can use the SimpleImputer in snowflake.ml.preprocessing to replace missing values with the most frequent.

In [29]:
my_imputer = SimpleImputer(input_cols=['OCCUPATION_TYPE'], output_cols=['OCCUPATION_TYPE'] ,strategy='most_frequent')
my_imputer.fit(snowpark_df)
snowpark_df = my_imputer.transform(snowpark_df)
snowpark_df.describe().show()

--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|"SUMMARY"  |"OCCUPATION_TYPE"     |"ID"               |"CODE_GENDER"  |"FLAG_OWN_CAR"  |"FLAG_OWN_REALTY"  |"CNT_CHILDREN"      |"AMT_INCOME_TOTAL"  |"NAME_INCOME_TYPE"    |"NAME_EDUCATION_TYPE"          |"NAME_FAMILY_STATUS"  |"NAME_HOUSING_TYPE"  |"DAYS_BIRTH"       |"DAYS_EMPLOYED"     |"FLAG_MOBIL"  |"FLAG_WORK_PHONE"   |"FLAG_PHONE"         |"FLAG_EMAIL"        |"CNT_FAM_MEMBERS"   |
----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

The missing value handling logic is converted in SQL to *iff("OCCUPATION_TYPE" IS NULL, 'Laborers', "OCCUPATION_TYPE") AS "OCCUPATION_TYPE"*

In [30]:
print(snowpark_df.queries['queries'][0])

SELECT iff("OCCUPATION_TYPE" IS NULL, 'Laborers', "OCCUPATION_TYPE") AS "OCCUPATION_TYPE", "ID", "CODE_GENDER", "FLAG_OWN_CAR", "FLAG_OWN_REALTY", "CNT_CHILDREN", "AMT_INCOME_TOTAL", "NAME_INCOME_TYPE", "NAME_EDUCATION_TYPE", "NAME_FAMILY_STATUS", "NAME_HOUSING_TYPE", "DAYS_BIRTH", "DAYS_EMPLOYED", "FLAG_MOBIL", "FLAG_WORK_PHONE", "FLAG_PHONE", "FLAG_EMAIL", "CNT_FAM_MEMBERS" FROM ( SELECT "ID", "CODE_GENDER", "FLAG_OWN_CAR", "FLAG_OWN_REALTY", "CNT_CHILDREN", "AMT_INCOME_TOTAL", "NAME_INCOME_TYPE", "NAME_EDUCATION_TYPE", "NAME_FAMILY_STATUS", "NAME_HOUSING_TYPE", "DAYS_BIRTH", "DAYS_EMPLOYED", "FLAG_MOBIL", "FLAG_WORK_PHONE", "FLAG_PHONE", "FLAG_EMAIL", "CNT_FAM_MEMBERS", "OCCUPATION_TYPE" AS "OCCUPATION_TYPE" FROM ( SELECT "ID", "CODE_GENDER", "FLAG_OWN_CAR", "FLAG_OWN_REALTY", "CNT_CHILDREN", "AMT_INCOME_TOTAL", "NAME_INCOME_TYPE", "NAME_EDUCATION_TYPE", "NAME_FAMILY_STATUS", "NAME_HOUSING_TYPE", "DAYS_BIRTH", "DAYS_EMPLOYED", "FLAG_MOBIL", "FLAG_WORK_PHONE", "FLAG_PHONE", "FLAG_EMA

A DataFrame always has a schema with all columns and the data types for them

In [31]:
snowpark_df.schema

StructType([StructField('OCCUPATION_TYPE', StringType(16777216), nullable=True), StructField('ID', LongType(), nullable=True), StructField('CODE_GENDER', StringType(16777216), nullable=True), StructField('FLAG_OWN_CAR', StringType(16777216), nullable=True), StructField('FLAG_OWN_REALTY', StringType(16777216), nullable=True), StructField('CNT_CHILDREN', LongType(), nullable=True), StructField('AMT_INCOME_TOTAL', DoubleType(), nullable=True), StructField('NAME_INCOME_TYPE', StringType(16777216), nullable=True), StructField('NAME_EDUCATION_TYPE', StringType(16777216), nullable=True), StructField('NAME_FAMILY_STATUS', StringType(16777216), nullable=True), StructField('NAME_HOUSING_TYPE', StringType(16777216), nullable=True), StructField('DAYS_BIRTH', LongType(), nullable=True), StructField('DAYS_EMPLOYED', LongType(), nullable=True), StructField('FLAG_MOBIL', LongType(), nullable=True), StructField('FLAG_WORK_PHONE', LongType(), nullable=True), StructField('FLAG_PHONE', LongType(), nullabl

We can itirate through the schema to get the columns of specific data types

In [32]:
# Get all categorical columns
categorical_types = [T.StringType]
categorical_columns = [c.name for c in snowpark_df.schema.fields if type(c.datatype) in categorical_types]
categorical_columns

['OCCUPATION_TYPE',
 'CODE_GENDER',
 'FLAG_OWN_CAR',
 'FLAG_OWN_REALTY',
 'NAME_INCOME_TYPE',
 'NAME_EDUCATION_TYPE',
 'NAME_FAMILY_STATUS',
 'NAME_HOUSING_TYPE']

We can then use those to loop through and get the number of unique values

In [33]:
# Number of unique values per categorical column
unique_values = []
for column in categorical_columns:
    unique_values.append([column, snowpark_df.select(column).distinct().count()])
pd.DataFrame(unique_values, columns=['COLUMN_NAME','NUM_UNIQUE_VALUES'])

,COLUMN_NAME,NUM_UNIQUE_VALUES
0,OCCUPATION_TYPE,18
1,CODE_GENDER,2
2,FLAG_OWN_CAR,2
3,FLAG_OWN_REALTY,2
4,NAME_INCOME_TYPE,5
5,NAME_EDUCATION_TYPE,5
6,NAME_FAMILY_STATUS,5
7,NAME_HOUSING_TYPE,6


## Persist Transformations

If we want to save the changes we can either save it as a table, meaning the SQL generated by the DataFrame is executed and the result is stored in a table or as a view where the DataFrame SQL will be the definition of the view.  
**save_as_table** saves the result in a table, if **mode='overwrite'** then it will also replace the data that is in it.

In [34]:
snowpark_df.write.save_as_table(table_name='MY_FIRST_ANALYSIS', mode='overwrite')
session.table('MY_FIRST_ANALYSIS').show()

----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|"OCCUPATION_TYPE"  |"ID"     |"CODE_GENDER"  |"FLAG_OWN_CAR"  |"FLAG_OWN_REALTY"  |"CNT_CHILDREN"  |"AMT_INCOME_TOTAL"  |"NAME_INCOME_TYPE"    |"NAME_EDUCATION_TYPE"          |"NAME_FAMILY_STATUS"  |"NAME_HOUSING_TYPE"  |"DAYS_BIRTH"  |"DAYS_EMPLOYED"  |"FLAG_MOBIL"  |"FLAG_WORK_PHONE"  |"FLAG_PHONE"  |"FLAG_EMAIL"  |"CNT_FAM_MEMBERS"  |
------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [35]:
session.close()